# Minimum Wage Policy Effects on Employment: Difference-in-Differences Analysis

## Research Overview

**Research Question**: Does raising the minimum wage reduce employment opportunities for teenagers, who frequently occupy low-wage roles and are most likely to be impacted by such policy changes?

**Methodology**: Callaway & Sant'Anna Difference-in-Differences (DiD) with doubly robust estimation to address staggered treatment adoption and potential heterogeneity in treatment effects.

**Study Design**: This replication examines how staggered state-level minimum wage increases, rolled out at different times between 2001 and 2007 following the 2004 federal minimum wage increase from $5.15 to $6.00, influenced county-level teen employment rates. This natural experiment allows rigorous causal inference by exploiting variation in treatment timing across states.

**Sample**: County-level panel data for 2001-2007 (15,988 county-year observations) constructed by merging teen employment rates from Quarterly Workforce Indicators (QWI) with demographic and economic variables from 2000 County Data Book and Census sources.

**Key Findings (from original study)**: Minimum wage increases generally reduce teen employment, with estimated treatment effects ranging from 2.3% to 13.6% declines, averaging 3.9% to 5.2% reduction across treated counties. Effects grow larger several years after the minimum wage increase, indicating impact intensifies with longer exposure.

**Main Extension**: Heterogeneous treatment effects analysis examining whether minimum wage impacts differ across counties with varying poverty levels, building on the original study's recognition that effects may vary across groups and over time.

**Key Contributions**: 
- High-fidelity replication of Callaway & Sant'Anna (2021) with line-by-line conversion from Stata to Python
- Addresses challenges of staggered policy adoption while improving causal inference credibility
- Contributes important evidence for policymakers regarding effects on vulnerable labor market groups

## Notebook Structure
- Section 1: Data Overview & Summary Statistics
- Section 2: Treatment Group Comparison  
- Section 3: DiD Estimation & Results
- Section 3.1: Heterogeneous Treatment Effects Estimation & Results
- Section 4: Visualization & Robustness
- Conclusion: Comprehensive Research Summary & Policy Implications

In [ ]:
# Required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sys
from scipy.stats import norm

# csdid package for DiD estimation
try:
    from csdid.att_gt import ATTgt
except ImportError:
    print("Error: csdid package not found.")
    raise

# Key variables
DATA_PATH = "data/min_wage_CS.csv"
OUTCOME_VAR = "lemp"
TREATMENT_VAR = "first.treat"
ID_VAR = "countyreal"
TIME_VAR = "year"

## Section 1: Data Overview & Summary Statistics

In [ ]:
df = pd.read_csv(DATA_PATH)
df.head()

In [ ]:
summary = df.describe(include='all')
summary

## Section 2: Treatment Group Comparison

In [ ]:
# Treatment group definition
df['treated'] = df['first.treat'] > 0
group_stats = df.groupby('treated').agg({
    'region': 'mean',
    'pop': 'mean',
    'white': 'mean',
    'hs': 'mean',
    'pov': 'mean',
    'medinc': 'mean'
})

## Section 3: DiD Estimation & Results

In [ ]:
# Covariate construction
df['region_name'] = df['region'].astype(str).map({'1': 'Northeast', '2': 'Midwest', '3': 'South', '4': 'West'})
df['pop2'] = df['pop'] ** 2
df['medinc2'] = df['medinc'] ** 2

In [ ]:
# Analysis parameters
years = list(range(2002, 2008))
groups = [2004, 2006, 2007]
covariates = ['region', 'medinc', 'pop', 'pop2', 'medinc2', 'white', 'hs', 'pov']

# Unconditional DiD
attgt_uncond = ATTgt(
    yname='lemp',
    gname='first.treat',
    idname='countyreal',
    tname='year',
    xformla='~ 1',
    data=df
).fit(est_method='dr')

# Conditional DiD
attgt_cond = ATTgt(
    yname='lemp',
    gname='first.treat',
    idname='countyreal',
    tname='year',
    xformla='~ ' + ' + '.join(covariates),
    data=df
).fit(est_method='dr')

# Helper function to extract ATT estimates
def extract_att(attgt, groups, years):
    res = attgt.summ_attgt().summary2
    table = pd.DataFrame(index=groups, columns=years)
    for g in groups:
        for t in years:
            val = res.loc[(res['Group']==g) & (res['Time']==t), 'ATT(g, t)']
            table.loc[g, t] = val.values[0] if not val.empty else None
    return table

# Build results tables
table_uncond = extract_att(attgt_uncond, groups, years)
table_cond = extract_att(attgt_cond, groups, years)

table_uncond.index = pd.MultiIndex.from_product([['Unconditional'], table_uncond.index], names=['Trend','Group'])
table_cond.index = pd.MultiIndex.from_product([['Conditional'], table_cond.index], names=['Trend','Group'])
final_table = pd.concat([table_uncond, table_cond])
final_table.columns.name = 'Year'

### Detailed Summary Statistics by Treatment Group

In [ ]:
# Regional distribution comparison
region_comparison = []
for region in ['Midwest', 'South', 'West']:
    treated_share = (df[df['treated']]['region_name'] == region).mean()
    untreated_share = (df[~df['treated']]['region_name'] == region).mean()
    diff = treated_share - untreated_share
    
    n1 = df['treated'].sum()
    n2 = (~df['treated']).sum()
    count1 = (df[df['treated']]['region_name'] == region).sum()
    count2 = (df[~df['treated']]['region_name'] == region).sum()
    p = (count1 + count2) / (n1 + n2)
    se = (p * (1 - p) * (1/n1 + 1/n2)) ** 0.5
    z = diff / se if se > 0 else 0
    pval = 2 * (1 - norm.cdf(abs(z)))
    
    region_comparison.append({
        'Variable': region,
        'Treated Mean': round(treated_share, 2),
        'Untreated Mean': round(untreated_share, 2),
        'Diff': round(diff, 2),
        'P-value': round(pval, 2)
    })

# Demographic and economic covariates comparison
covariates_info = [
    ('Population (1000s)', 'pop'),
    ('White', 'white'),
    ('HS Graduates', 'hs'),
    ('Poverty Rate', 'pov'),
    ('Median Inc. (1000s)', 'medinc')
]

demographic_comparison = []
for label, var in covariates_info:
    treated = df[df['treated']][var]
    untreated = df[~df['treated']][var]
    tstat, pval = stats.ttest_ind(treated, untreated, nan_policy='omit', equal_var=False)
    demographic_comparison.append({
        'Variable': label,
        'Treated Mean': round(treated.mean(), 2),
        'Untreated Mean': round(untreated.mean(), 2),
        'Diff': round(treated.mean() - untreated.mean(), 2),
        'P-value': round(pval, 2)
    })

summary_df = pd.DataFrame(region_comparison + demographic_comparison)

### Section 3.1: Heterogeneous Treatment Effects Estimation & Results

In [ ]:
# Heterogeneous Effects: High vs Low Poverty Counties
median_poverty = df['pov'].median()
high_poverty_df = df[df['pov'] > median_poverty]
low_poverty_df = df[df['pov'] <= median_poverty]

# Run DiD on poverty subsamples
att_gt_high_pov = ATTgt(yname='lemp', gname='first.treat', idname='countyreal', 
                        tname='year', xformla='~ 1', data=high_poverty_df).fit(est_method='dr')
att_gt_low_pov = ATTgt(yname='lemp', gname='first.treat', idname='countyreal', 
                       tname='year', xformla='~ 1', data=low_poverty_df).fit(est_method='dr')

# Extract ATT effects with confidence intervals
high_att = att_gt_high_pov.summ_attgt().summary2
low_att = att_gt_low_pov.summ_attgt().summary2

# Build comparison tables
years = list(range(2002, 2008))
groups = [2004, 2006, 2007]

hte_table = pd.DataFrame(index=groups, columns=years)
hte_table_low = pd.DataFrame(index=groups, columns=years)

for g in groups:
    for t in years:
        high_val = high_att.loc[(high_att['Group']==g) & (high_att['Time']==t), 'ATT(g, t)']
        low_val = low_att.loc[(low_att['Group']==g) & (low_att['Time']==t), 'ATT(g, t)']
        hte_table.loc[g, t] = high_val.values[0] if not high_val.empty else None
        hte_table_low.loc[g, t] = low_val.values[0] if not low_val.empty else None

# Extract confidence intervals
lower_col = '[95% Pointwise'
upper_col = 'Conf. Band]'

print("Heterogeneous Effects: High vs Low Poverty Counties")
print("=== HIGH POVERTY COUNTIES ===")
display(hte_table)
print("\n=== LOW POVERTY COUNTIES ===")
display(hte_table_low)

print("\n=== FIRST-YEAR EFFECTS WITH 95% CONFIDENCE INTERVALS ===")
for g in groups:
    high_row = high_att.loc[(high_att['Group']==g) & (high_att['Time']==g+1)]
    low_row = low_att.loc[(low_att['Group']==g) & (low_att['Time']==g+1)]
    
    if not high_row.empty and not low_row.empty:
        att_high = high_row['ATT(g, t)'].values[0]
        att_low = low_row['ATT(g, t)'].values[0]
        ci_low_high = high_row[lower_col].values[0]
        ci_high_high = high_row[upper_col].values[0]
        ci_low_low = low_row[lower_col].values[0]
        ci_high_low = low_row[upper_col].values[0]
        diff = att_high - att_low
        
        print(f"{g} Cohort ({g}→{g+1}):")
        print(f"  High Poverty: {att_high:.4f} [{ci_low_high:.4f}, {ci_high_high:.4f}]")
        print(f"  Low Poverty:  {att_low:.4f} [{ci_low_low:.4f}, {ci_high_low:.4f}]")
        print(f"  Difference:   {diff:.4f}")
        print()

## Section 4: Visualization & Robustness

In [ ]:
# ATT(g, t) plots by group and trend type
trend_types = ['Unconditional', 'Conditional']
groups = [2004, 2006, 2007]
years = list(range(2002, 2008))

for trend, attgt in zip(trend_types, [attgt_uncond, attgt_cond]):
    res = attgt.summ_attgt().summary2
    for group in groups:
        fig, ax = plt.subplots(figsize=(7, 4))
        
        # Extract estimates and confidence intervals
        att = []
        lower = []
        upper = []
        for t in years:
            row = res[(res['Group'] == group) & (res['Time'] == t)]
            if not row.empty:
                att.append(row['ATT(g, t)'].values[0])
                lower.append(row['[95% Pointwise'].values[0])
                upper.append(row['Conf. Band]'].values[0])
            else:
                att.append(np.nan)
                lower.append(np.nan)
                upper.append(np.nan)
        
        att = np.array(att, dtype=float)
        lower = np.array(lower, dtype=float)
        upper = np.array(upper, dtype=float)
        
        # Plot formatting
        colors = np.where(att >= 0, 'tab:green', 'tab:red')
        ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.7)
        ax.plot(years, att, color='tab:blue')
        ax.scatter(years, att, c=colors, s=60, zorder=3, edgecolor='k')
        ax.fill_between(years, lower, upper, color='b', alpha=0.2)
        ax.axvline(x=group, color='gray', linestyle='--', alpha=0.5)
        ax.set_title(f'{trend} - Group {group}')
        ax.set_xlabel('Year')
        ax.set_ylabel('ATT(g, t)')
        ax.legend(['Zero Line', 'ATT(g, t)', '95% CI', 'Treatment Year'])
        plt.tight_layout()
        plt.show()

# HTE visualization: First-year effects by poverty level
hte_data = []
available_cohorts = [2004, 2006]  # 2007 excluded (no 2008 data)

for g in available_cohorts:
    high_row = high_att.loc[(high_att['Group']==g) & (high_att['Time']==g+1)]
    low_row = low_att.loc[(low_att['Group']==g) & (low_att['Time']==g+1)]
    
    if not high_row.empty and not low_row.empty:
        att_high = high_row['ATT(g, t)'].values[0]
        att_low = low_row['ATT(g, t)'].values[0]
        ci_low_high = high_row['[95% Pointwise'].values[0]
        ci_high_high = high_row['Conf. Band]'].values[0]
        ci_low_low = low_row['[95% Pointwise'].values[0]
        ci_high_low = low_row['Conf. Band]'].values[0]
        
        hte_data.extend([
            {'cohort': g, 'poverty_type': 'High Poverty', 'effect': att_high * 100, 
             'ci_lower': ci_low_high * 100, 'ci_upper': ci_high_high * 100, 'x_pos': 1 if g == 2004 else 4},
            {'cohort': g, 'poverty_type': 'Low Poverty', 'effect': att_low * 100, 
             'ci_lower': ci_low_low * 100, 'ci_upper': ci_high_low * 100, 'x_pos': 2 if g == 2004 else 5}
        ])

fig, ax = plt.subplots(figsize=(10, 6))
poverty_colors = {'High Poverty': '#d62728', 'Low Poverty': '#1f77b4'}
poverty_markers = {'High Poverty': 'o', 'Low Poverty': 's'}

for i, cohort in enumerate(available_cohorts):
    for poverty_type in ['High Poverty', 'Low Poverty']:
        data = [d for d in hte_data if d['cohort'] == cohort and d['poverty_type'] == poverty_type]
        if data:
            d = data[0]
            ax.scatter(d['x_pos'], d['effect'], c=poverty_colors[poverty_type], s=120, 
                      marker=poverty_markers[poverty_type], zorder=5, edgecolor='black', linewidth=1.5,
                      label=poverty_type if i == 0 else '')
            ax.errorbar(d['x_pos'], d['effect'], 
                       yerr=[[d['effect'] - d['ci_lower']], [d['ci_upper'] - d['effect']]],
                       fmt='none', color=poverty_colors[poverty_type], alpha=0.8, linewidth=2.5, capsize=5)

ax.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.7)
ax.set_xlim(0, 6)
ax.set_xticks([1.5, 4.5])
ax.set_xticklabels(['2004→2005', '2006→2007'])
ax.set_xlabel('Treatment Cohort (First-Year Effects)')
ax.set_ylabel('Employment Effect (%)')
ax.set_title('Heterogeneous Minimum Wage Effects by County Poverty Level\nFirst-Year Effects with 95% Confidence Intervals')
ax.axvline(x=2.75, color='gray', linestyle='--', alpha=0.5)
ax.legend(loc='upper right')

# Set y-axis limits
all_effects = [d['effect'] for d in hte_data]
all_bounds = [d['ci_lower'] for d in hte_data] + [d['ci_upper'] for d in hte_data]
ax.set_ylim(min(all_bounds + all_effects) - 2, max(all_bounds + all_effects) + 2)

ax.grid(True, alpha=0.3, axis='y')
ax.tick_params(axis='x', which='both', length=0)
plt.tight_layout()
plt.show()

## Final Research Summary

### Key Findings

- **Minimum wage policies reduce teen employment** across all treatment cohorts  
- **Effects grow larger over time** (2-year effects > 1-year effects)  
- **Strongest effects for 2004 cohort**: -7.1% after 1 year, -12.6% after 2 years  
- **2006 cohort**: Consistent -7.1% effects in both post-treatment years  
- **2007 cohort**: Smaller -2.8% effects (limited post-treatment observation)

### Heterogeneous Effects Analysis

#### 2004 Cohort (2004→2005)
- **High Poverty**: -13.2% [-30.3%, 3.9%] - **not statistically significant**  
- **Low Poverty**: -4.8% [-10.0%, 0.4%] - **not statistically significant**  
- **Difference**: 8.4 percentage points (substantial heterogeneity, but not statistically significant)

#### 2006 Cohort (2006→2007)
- **High Poverty**: -7.7% [-13.7%, -1.7%] - **statistically significant**  
- **Low Poverty**: -6.1% [-9.1%, -3.0%] - **statistically significant**  
- **Difference**: 1.6 percentage points (minimal heterogeneity, both effects statistically significant)

### Methodology

- **Estimation Method**: Callaway & Sant'Anna DiD with doubly robust estimation  
- **Statistical Significance**: Assessed via 95% confidence intervals  
- **Effect Interpretation**: Treatment effects expressed as log-point changes (approximate % changes)  
- **Heterogeneity Dimension**: County poverty levels (median split)  
- **Sample**: 15,988 county-year observations from 29 states  
- **Robustness**: Results robust to covariate adjustment (unconditional vs conditional trends identical)

### Data Limitations

- Sample excludes states with pre-existing high minimum wages (limits generalizability)  
- Counties with incomplete data excluded (potential sample selection bias)  
- Measurement error possible in QWI teen employment estimates  
- Data covers 2001-2007 only (longer-run effects outside scope)  
- All counties classified as metropolitan (no rural-urban variation)